In [1]:
from transformer_heads import load_headed
from datasets import load_dataset
from transformers import (
    AutoTokenizer,
    MistralForCausalLM,
    Trainer,
    BitsAndBytesConfig,
    TrainingArguments,
)
from transformer_heads.util.helpers import DataCollatorWithPadding
from peft import LoraConfig
from transformer_heads.config import HeadConfig
from transformer_heads.util.model import print_trainable_parameters
import torch

In [2]:
model_class, model_path = MistralForCausalLM, "mistralai/Mistral-7B-v0.1"

In [3]:
heads_configs = [
    HeadConfig(
        name="wikitext_head",
        layer_hook=-1,  # Hook to layer [-4] (Drop 3 layers from the end)
        in_size=4096,
        hidden_size=0,
        num_layers=1,
        output_activation="linear",
        is_causal_lm=True,
        loss_fct="cross_entropy",
        num_outputs=32000,
        is_regression=False,
        output_bias=False,
    )
]

In [4]:
dd = load_dataset("wikitext", "wikitext-2-v1")

In [5]:
tokenizer = AutoTokenizer.from_pretrained(model_path)
if tokenizer.pad_token_id is None:
    tokenizer.pad_token = tokenizer.eos_token


def tokenize_function(examples):
    out = tokenizer(examples["text"], padding="max_length", truncation=True)
    out[heads_configs[0].name] = out["input_ids"].copy()
    return out


for split in dd.keys():
    dd[split] = dd[split].map(tokenize_function, batched=True)
dd.set_format(
    type="torch", columns=["input_ids", "attention_mask", heads_configs[0].name]
)
for split in dd.keys():
    dd[split] = dd[split].remove_columns("text")

Map:   0%|          | 0/4358 [00:00<?, ? examples/s]

Asking to pad to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no padding.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Map:   0%|          | 0/36718 [00:00<?, ? examples/s]

Map:   0%|          | 0/3760 [00:00<?, ? examples/s]

In [6]:
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    load_in_8bit=False,
    llm_int8_threshold=6.0,
    llm_int8_has_fp16_weight=False,
    bnb_4bit_compute_dtype=torch.float32,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
)

model = load_headed(
    model_class,
    model_path,
    head_configs=heads_configs,
    quantization_config=quantization_config,
    device_map={"": torch.cuda.current_device()},
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Some weights of TransformerWithHeads were not initialized from the model checkpoint at mistralai/Mistral-7B-v0.1 and are newly initialized: ['heads.wikitext_head.lins.0.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [7]:
print_trainable_parameters(model)

all params: 3752071168 || trainable params: 131072000 || trainable%: 3.493323930469754
params by dtype: defaultdict(<class 'int'>, {torch.float32: 262410240, torch.uint8: 3489660928})
trainable params by dtype: defaultdict(<class 'int'>, {torch.float32: 131072000})


In [8]:
dd["train"]

Dataset({
    features: ['input_ids', 'attention_mask', 'wikitext_head'],
    num_rows: 36718
})

In [9]:
args = TrainingArguments(
    output_dir="linear_probe_test",
    learning_rate=0.0002,
    num_train_epochs=0.1,
    logging_steps=5,
    do_eval=False,
    remove_unused_columns=False,
)
collator = DataCollatorWithPadding(
    feature_name_to_padding_value={
        "input_ids": tokenizer.pad_token_id,
        heads_configs[0].name: -100,
        "attention_mask": 0,
    }
)
trainer = Trainer(
    model,
    args=args,
    train_dataset=dd["train"],
    eval_dataset=dd["validation"],
    data_collator=collator,
)
trainer.train()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: ykeller (chm-hci). Use `wandb login --relogin` to force relogin


`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
/opt/miniconda3/envs/sh_finetune/lib/python3.10/site-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss
5,28.830500
10,24.946500
15,22.368700
20,19.769200
25,17.926200
30,17.926900
35,15.915800
40,14.250100
45,15.602900
50,15.518000


TrainOutput(global_step=459, training_loss=11.4218144551861, metrics={'train_runtime': 1066.8159, 'train_samples_per_second': 3.442, 'train_steps_per_second': 0.43, 'total_flos': 3.991059921626726e+16, 'train_loss': 11.4218144551861, 'epoch': 0.1})

In [10]:
trainer.evaluate()

{'eval_runtime': 328.8245,
 'eval_samples_per_second': 11.435,
 'eval_steps_per_second': 1.429,
 'epoch': 0.1}

In [17]:
input = tokenizer("The historical significance of", return_tensors="pt")
output = model(**input)
logits = output.logits_by_head[heads_configs[0].name]
predicted_ids = torch.argmax(logits, dim=-1)
print(tokenizer.decode(predicted_ids[0], skip_special_tokens=True))

 group Fran of the
